In [1]:
import os, sys
sys.path.append("..")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np
import torch
import torch.nn as nn
import torchvision
import gc

from src.tools import unfreeze, freeze
from src.tools import load_dataset, get_loader_stats

from copy import deepcopy
import json

from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

# This needed to use dataloaders for some datasets
from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

In [2]:
gc.collect(); torch.cuda.empty_cache()

## Main Config

In [3]:
DEVICE_ID = 0

# DATASET_LIST = [
#     ('MNIST-colored_3', '/gpfs/data/gpfs0/n.gushchin/MNIST', 32, 1),
# ]
DATASET_LIST = [
    ('CelebA_high', '/gpfs/data/gpfs0/n.gushchin/img_align_celeba', 64, 1),
]

assert torch.cuda.is_available()
torch.cuda.set_device(f'cuda:{DEVICE_ID}')

In [4]:
for DATASET, DATASET_PATH, IMG_SIZE, N_EPOCHS in tqdm(DATASET_LIST):
    print('Processing {}'.format(DATASET))
    sampler, test_sampler = load_dataset(DATASET, DATASET_PATH, img_size=IMG_SIZE, batch_size=256)
    print('Dataset {} loaded'.format(DATASET))

    mu, sigma = get_loader_stats(test_sampler.loader, n_epochs=N_EPOCHS, verbose=True, batch_size=256)
    print('Trace of sigma: {}'.format(np.trace(sigma)))
    stats = {'mu' : mu.tolist(), 'sigma' : sigma.tolist()}
    print('Stats computed')

    filename = '{}_{}_test.json'.format(DATASET, IMG_SIZE)
    with open(filename, 'w') as fp:
        json.dump(stats, fp)
    print('States saved to {}'.format(filename))

<ipython-input-4-d34b623c3c52>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for DATASET, DATASET_PATH, IMG_SIZE, N_EPOCHS in tqdm(DATASET_LIST):


  0%|          | 0/1 [00:00<?, ?it/s]

Processing CelebA_high
Dataset CelebA_high loaded


/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


0it [00:00, ?it/s]

Trace of sigma: 114.5528501160176
Stats computed
States saved to CelebA_high_64_test.json
